In [1]:
#task 1
from collections import Counter

count_dict = Counter(list(open('text.txt').read().lower()))
sorted_dict = {}

for symbol, count in count_dict.items():
    if symbol.isalpha():
        sorted_dict[symbol] = count
        
print(dict(sorted(sorted_dict.items(), reverse = True, key=lambda x: x[1])))

{'в': 6, 'е': 4, 'ь': 3, 'п': 2, 'о': 2, 'р': 1, 'а': 1, 'г': 1}


In [2]:
#task 2
import hashlib
import os
# директория для поиска файлов-дубликатов
directory = "/path/to/directory"

# Считаем все файлы в данной директории и в ее подпапках
files = []
for dirpath, dirnames, filenames in os.walk(directory):
    for filename in filenames:
        files.append(os.path.join(dirpath, filename))

# Вычислим хэш-сумму каждого файла
hashes = {}
for file in files:
    with open(file, "rb") as f:
        data = f.read()
        hash = hashlib.md5(data).hexdigest()

    # Создадим словарь, где ключами будут являться хэш-суммы файлов, 
    # а значениями - списки файлов с этой хэш-суммой
    if hash in hashes:
        hashes[hash].append(file)
    else:
        hashes[hash] = [file]

# Выведем на экран группы имен файлов-дубликатов.
for hash, files in hashes.items():
    if len(files) > 1:
        print("Дубликаты файлов с хэш-суммой", hash)
        for file in files:
            print(file)
        print("")

In [ ]:
#task 3
import os

# Путь к директории с музыкальными файлами
music_dir = r"/directory"

# Путь к текстовому файлу со списком песен
playlist_file = "/path/to/playlist.txt"

# Считываем список песен из текстового файла
with open(playlist_file, "r") as f:
    songs = f.readlines()

# Обходим список песен и переименовываем соответствующие файлы в директории
for song in songs:
    # Извлекаем номер песни и название
    song_number, song_name = song.strip().split(". ", 1)

    # Ищем файл с соответствующим названием песни в директории
    for file_name in os.listdir(music_dir):
        if song_name in file_name:
            # Переименовываем файл, добавляя номер песни в начало имени
            new_file_name = f"{song_number}.{file_name}"
            os.rename(os.path.join(music_dir, file_name), os.path.join(music_dir, new_file_name))
            print(f"Переименован файл {file_name} в {new_file_name}")
            break
    

In [ ]:
#task 4
import re

# Ввод имени файла с клавиатуры
filename = input("Введите имя файла: ")

# Открываем файл и считываем его содержимое
with open(filename, "r") as f:
    content = f.read()

# Определяем регулярное выражение для поиска строк заданного вида
pattern = r"\w+: (int|short|byte) \[\d+\]"

# Ищем все подстроки, соответствующие заданному регулярному выражению
matches = re.findall(pattern, content)

# Выводим найденные строки
for match in matches:
    print(match)

In [61]:
#task 5
import re
s = "kovsd Petr34 niorv39443 Ivan2002"
match = re.findall(r'\b[A-Z][a-z]+\d{2}(?:\d{2})?\b', s)
print(match)

['Petr34', 'Ivan2002']


In [ ]:
#task 6
import os
import shutil
import datetime

def create_directories(source_dir, days, size):
    # Получение текущей даты
    current_date = datetime.datetime.now()

    # Поиск файлов для перемещения в директорию Archive
    archive_files = []
    for filename in os.listdir(source_dir):
        filepath = os.path.join(source_dir, filename)
        modified_date = datetime.datetime.fromtimestamp(os.path.getmtime(filepath))
        if (current_date - modified_date).days > days:
            archive_files.append(filename)

    # Перемещение файлов в директорию Archive
    if archive_files:
        archive_dir = os.path.join(source_dir, 'Archive')
        if not os.path.exists(archive_dir):
            os.makedirs(archive_dir)
        for filename in archive_files:
            filepath = os.path.join(source_dir, filename)
            shutil.move(filepath, os.path.join(archive_dir, filename))

    # Поиск файлов для перемещения в директорию Small
    small_files = []
    for filename in os.listdir(source_dir):
        filepath = os.path.join(source_dir, filename)
        if os.path.getsize(filepath) < size:
            small_files.append(filename)

    # Перемещение файлов в директорию Small
    if small_files:
        small_dir = os.path.join(source_dir, 'Small')
        if not os.path.exists(small_dir):
            os.makedirs(small_dir)
        for filename in small_files:
            filepath = os.path.join(source_dir, filename)
            shutil.move(filepath, os.path.join(small_dir, filename))


# Пример вызова функции
source_dir = './source'
days = 30
size = 1024
create_directories(source_dir, days, size)

In [ ]:
#task 7
import os
import subprocess
import random

def create_track_mix(source_dir, destination_file=None, count=None, frame=None, log=False, extended=False):
    # формируем список mp3-файлов в указанной директории
    file_list = []
    for file in os.listdir(source_dir):
        if file.endswith(".mp3"):
            file_list.append(os.path.join(source_dir, file))

    # если не указано количество файлов в "нарезке", то перебираем все файлы
    if not count:
        count = len(file_list)

    # если не указано количество секунд на каждый файл, то вырезаем по 10 секунд из произвольного участка каждого файла
    if not frame:
        frame = 10

    # если не указано имя выходного файла, то сохраняем в mix.mp3 в директории source
    if not destination_file:
        destination_file = os.path.join(source_dir, "mix.mp3")

    # формируем список для ffmpeg
    mix_list = []
    for i in range(count):
        file = random.choice(file_list)
        start = random.randint(0, 60 - frame)
        mix_list.extend(["-ss", str(start), "-t", str(frame), "-i", file])

        # добавляем fade in/fade out если указан параметр extended
        if extended:
            mix_list.extend(["-af", f"afade=t=in:st=0:d=1,afade=t=out:st={frame - 1}:d=1"])

        # добавляем лог если указан параметр log
        if log:
            print(f"--- processing file {i + 1}: {os.path.basename(file)}")

    # склеиваем файлы
    cmd = ["./ffmpeg/bin/ffmpeg", "-y"] + mix_list + ["-filter_complex", "amix=inputs={}:duration=first".format(count), destination_file]
    subprocess.call(cmd)

    if log:
        print("--- done!")
create_track_mix(source_dir="./Album", count=2, frame=47, log=True, extended=True)